In [1]:
import os
import pandas as pd

SCORE_DIR = "data/scores"

In [2]:
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [3]:
from bs4 import BeautifulSoup

def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [4]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [5]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score

In [6]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [10]:
pip install html5lib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 1.3 MB/s eta 0:00:00 0:00:01
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [12]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

/tmp/ipykernel_684/2138878849.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
/tmp/ipykernel_684/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
/tmp/ipykernel_684/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
/tmp/ipykernel_684/846252781.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future ver

In [13]:
games_df = pd.concat(games, ignore_index=True)

In [14]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,39.0,85.0,0.459,9.0,25.0,0.360,21.0,26.0,...,28.6,30.6,183.0,116.0,MIN,106,1,2024,2024-02-02,True
1,240.0,240.0,38.0,82.0,0.463,12.0,38.0,0.316,18.0,20.0,...,50.0,29.8,159.0,117.0,ORL,108,0,2024,2024-02-02,False
2,240.0,240.0,46.0,83.0,0.554,11.0,29.0,0.379,18.0,22.0,...,22.2,38.5,159.0,129.0,MEM,101,1,2024,2024-02-02,True
3,240.0,240.0,35.0,87.0,0.402,14.0,46.0,0.304,17.0,24.0,...,36.4,33.3,206.0,113.0,GSW,121,0,2024,2024-02-02,False
4,240.0,240.0,43.0,93.0,0.462,17.0,44.0,0.386,7.0,12.0,...,100.0,29.5,276.0,123.0,CLE,136,1,2024,2024-02-05,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,240.0,240.0,38.0,91.0,0.418,8.0,30.0,0.267,11.0,12.0,...,25.0,37.3,167.0,112.0,LAC,103,0,2024,2024-02-04,False
70,240.0,240.0,39.0,80.0,0.488,12.0,31.0,0.387,23.0,27.0,...,11.1,34.6,206.0,127.0,NYK,105,1,2024,2024-02-03,True
71,240.0,240.0,41.0,100.0,0.410,10.0,35.0,0.286,13.0,15.0,...,16.1,28.9,206.0,126.0,LAL,113,0,2024,2024-02-03,False
72,265.0,265.0,50.0,112.0,0.446,18.0,56.0,0.321,16.0,20.0,...,11.5,26.0,175.0,124.0,ATL,141,1,2024,2024-02-03,False


In [15]:
games_df.to_csv("nba_games.csv")